# **Feature Extraction Methods: Imbalanced Data With Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random
import IPython.display as ipd
from functools import partial

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(annotated_train_csv)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

# **Creating a class to do the extraction**

In [4]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=60, n_mfcc=20, n_chroma=12, features=['mfcc'], normalize=True, avgpool=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'rms', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      maxpool (bool): Whether to maxpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'rms', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'rms', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features = self.feature_extraction(self.val_df, window_size=self.window_size)

    # process the features by average pooling
    self.train_features, self.val_features = self.process_features(self.train_features, self.val_features)

    
  

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def bandpass_filter(self, audio, lowcut=800, highcut=8000, order=4):
    nyquist = 0.5 * self.sr  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist

    b, a = butter(order, [low, high], btype='band')
    filtered_audio = filtfilt(b, a, audio)
    return filtered_audio
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_rms(self, window):
    return librosa.feature.rms(y=window)

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels)
    if self.normalize:
      return librosa.util.normalize(mel)
    else:
      return mel
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    for each in train_features_dict.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'rms':
        n_features=1
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_features_dict[each]
      val_feature = val_features_dict[each]

      if self.avgpool:
        train_features_dict[each], val_features_dict[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_features_dict[each], val_features_dict[each] = train_features_dict[each], val_features_dict[each]
    
    return train_features_dict, val_features_dict
      

  def feature_extraction(self, dataframe, window_size, filter=True):
    y = [] # To hold the labels
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])
          start = row['start']
          end = row['end']

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          start = int(start * sr)
          end = int(end * sr)+512



          if end > len(audio):
             end = len(audio)

          sample = audio[start:end]

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          # print(len(sample))

          if filter:
                  sample = self.bandpass_filter(sample)

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)

          # If not using average pooling, return resized features
    return y, features_dict

# **Average Pooling**

## **Window Size = 1s**

### **['melspectrogram']**

In [5]:
features_list = ['melspectrogram']

In [6]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (3444, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 3444


Processing data: 100%|██████████| 3444/3444 [01:00<00:00, 56.71it/s] 


Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [00:13<00:00, 60.29it/s] 



393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step


In [7]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(12565,)

'melspectrogram'

(12565, 60)

In [8]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(3318,)

'melspectrogram'

(3318, 60)

### Encode Classes

In [9]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [ 9  9  9 ... 17 17 17]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [10]:
display(len(train_y_encoded))
display(train_y_encoded[:10])

display(len(val_y_encoded))
display(val_y_encoded[:10])

12565

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

3318

array([14, 14, 14, 14, 14, 14, 10, 10, 10, 10])

In [11]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [12]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[9.4866091e-06, 9.5487494e-06, 9.6551566e-06, ..., 9.8847534e-07,
          9.3044491e-07, 9.0232516e-07],
         [1.2129851e-05, 1.2414495e-05, 1.2921279e-05, ..., 5.2223759e-06,
          4.9851474e-06, 4.8430848e-06],
         [5.8306232e-05, 5.9333925e-05, 6.1116712e-05, ..., 1.7024886e-06,
          1.6033545e-06, 1.5549887e-06],
         ...,
         [1.2095724e-04, 1.2229703e-04, 1.2461811e-04, ..., 2.1898311e-06,
          1.1668556e-06, 1.0314271e-06],
         [1.9102510e-05, 1.9464049e-05, 2.0094065e-05, ..., 4.9249811e-06,
          4.0005261e-06, 3.6747458e-06],
         [4.3214677e-05, 4.4297904e-05, 4.6190373e-05, ..., 1.4690548e-06,
          7.8608684e-07, 7.2303249e-07]], dtype=float32),
  'label': array([ 9,  9,  9, ..., 17, 17, 17])},
 'val': {'melspectrogram': array([[4.8846392e-05, 4.9467966e-05, 5.0583865e-05, ..., 2.4733069e-06,
          8.2670647e-07, 7.3230768e-07],
         [3.8189260e-06, 3.7921482e-06, 3.7431978e-06, 

### Save the merged dictionary to a pkl

In [13]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/Annotated/AveragePooled/split_features_1s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [14]:
features_list = ['melspectrogram', 'mfcc']

In [15]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (3444, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 3444


Processing data: 100%|██████████| 3444/3444 [01:43<00:00, 33.17it/s]


Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [00:27<00:00, 30.48it/s]


393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step


In [16]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(12565,)

'melspectrogram'

(12565, 60)

'mfcc'

(12565, 20)

In [17]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(3318,)

'melspectrogram'

(3318, 60)

'mfcc'

(3318, 20)

### Encode Classes

In [18]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [ 9  9  9 ... 17 17 17]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [19]:
display(len(train_y_encoded))
display(train_y_encoded[:10])

display(len(val_y_encoded))
display(val_y_encoded[:10])

12565

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

3318

array([14, 14, 14, 14, 14, 14, 10, 10, 10, 10])

In [20]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [21]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[9.4866091e-06, 9.5487494e-06, 9.6551566e-06, ..., 9.8847534e-07,
          9.3044491e-07, 9.0232516e-07],
         [1.2129851e-05, 1.2414495e-05, 1.2921279e-05, ..., 5.2223759e-06,
          4.9851474e-06, 4.8430848e-06],
         [5.8306232e-05, 5.9333925e-05, 6.1116712e-05, ..., 1.7024886e-06,
          1.6033545e-06, 1.5549887e-06],
         ...,
         [6.9514776e-06, 6.9201660e-06, 6.8409195e-06, ..., 2.1560011e-06,
          1.2065759e-06, 1.0922661e-06],
         [1.9102510e-05, 1.9464049e-05, 2.0094065e-05, ..., 4.9249811e-06,
          4.0005261e-06, 3.6747458e-06],
         [5.4058899e-05, 5.5537719e-05, 5.8154397e-05, ..., 9.8274654e-07,
          4.5612131e-07, 4.3058299e-07]], dtype=float32),
  'mfcc': array([[-1.        , -0.03889354, -0.64160925, ..., -0.02376524,
           0.02349628, -0.00874254],
         [-1.        , -0.0368154 , -0.633421  , ..., -0.02121266,
           0.0290794 , -0.01421679],
         [-1.        , -0.0284

### Save the merged dictionary to a pkl

In [22]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/Annotated/AveragePooled/split_features_1s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [23]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [24]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (3444, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 3444


Processing data:  35%|███▍      | 1199/3444 [01:02<01:05, 34.42it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 3444/3444 [02:48<00:00, 20.46it/s]


Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [00:42<00:00, 19.41it/s]


393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step


In [25]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(12565,)

'melspectrogram'

(12565, 60)

'mfcc'

(12565, 20)

'chroma'

(12565, 12)

In [26]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(3318,)

'melspectrogram'

(3318, 60)

'mfcc'

(3318, 20)

'chroma'

(3318, 12)

### Encode Classes

In [27]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [ 9  9  9 ... 17 17 17]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [28]:
display(len(train_y_encoded))
display(train_y_encoded[:10])

display(len(val_y_encoded))
display(val_y_encoded[:10])

12565

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

3318

array([14, 14, 14, 14, 14, 14, 10, 10, 10, 10])

In [29]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [30]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[9.4866091e-06, 9.5487494e-06, 9.6551566e-06, ..., 9.8847534e-07,
          9.3044491e-07, 9.0232516e-07],
         [1.2129851e-05, 1.2414495e-05, 1.2921279e-05, ..., 5.2223759e-06,
          4.9851474e-06, 4.8430848e-06],
         [5.8306232e-05, 5.9333925e-05, 6.1116712e-05, ..., 1.7024886e-06,
          1.6033545e-06, 1.5549887e-06],
         ...,
         [8.7809662e-05, 8.9015433e-05, 9.1153270e-05, ..., 1.1500978e-06,
          4.3757257e-07, 4.0334126e-07],
         [1.9102510e-05, 1.9464049e-05, 2.0094065e-05, ..., 4.9249811e-06,
          4.0005261e-06, 3.6747458e-06],
         [3.4193312e-05, 3.5084158e-05, 3.6650839e-05, ..., 1.0402787e-06,
          5.3121545e-07, 5.1639358e-07]], dtype=float32),
  'mfcc': array([[-1.        , -0.03889354, -0.64160925, ..., -0.02376524,
           0.02349628, -0.00874254],
         [-1.        , -0.0368154 , -0.633421  , ..., -0.02121266,
           0.0290794 , -0.01421679],
         [-1.        , -0.0284

### Save the merged dictionary to a pkl

In [31]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/Annotated/AveragePooled/split_features_1s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [32]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [33]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (3444, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 3444


Processing data:  35%|███▍      | 1201/3444 [01:06<01:04, 34.95it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 3444/3444 [03:04<00:00, 18.63it/s]


Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [00:48<00:00, 17.26it/s]


393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step
393/393 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step


In [34]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(12565,)

'melspectrogram'

(12565, 60)

'mfcc'

(12565, 20)

'chroma'

(12565, 12)

'rms'

(12565, 1)

In [35]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(3318,)

'melspectrogram'

(3318, 60)

'mfcc'

(3318, 20)

'chroma'

(3318, 12)

'rms'

(3318, 1)

### Encode Classes

In [36]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [ 9  9  9 ... 17 17 17]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [37]:
display(len(train_y_encoded))
display(train_y_encoded[:10])

display(len(val_y_encoded))
display(val_y_encoded[:10])

12565

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

3318

array([14, 14, 14, 14, 14, 14, 10, 10, 10, 10])

In [38]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [39]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[9.4866091e-06, 9.5487494e-06, 9.6551566e-06, ..., 9.8847534e-07,
          9.3044491e-07, 9.0232516e-07],
         [1.2129851e-05, 1.2414495e-05, 1.2921279e-05, ..., 5.2223759e-06,
          4.9851474e-06, 4.8430848e-06],
         [5.8306232e-05, 5.9333925e-05, 6.1116712e-05, ..., 1.7024886e-06,
          1.6033545e-06, 1.5549887e-06],
         ...,
         [1.7914819e-04, 1.8226101e-04, 1.8773414e-04, ..., 2.3687387e-06,
          1.6498615e-06, 1.5470791e-06],
         [1.9102510e-05, 1.9464049e-05, 2.0094065e-05, ..., 4.9249811e-06,
          4.0005261e-06, 3.6747458e-06],
         [9.0032081e-06, 9.2558430e-06, 9.7203119e-06, ..., 9.5587347e-07,
          4.5187107e-07, 4.2664561e-07]], dtype=float32),
  'mfcc': array([[-1.        , -0.03889354, -0.64160925, ..., -0.02376524,
           0.02349628, -0.00874254],
         [-1.        , -0.0368154 , -0.633421  , ..., -0.02121266,
           0.0290794 , -0.01421679],
         [-1.        , -0.0284

### Save the merged dictionary to a pkl

In [40]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/Annotated/AveragePooled/split_features_1s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)